# 使用FP-growth算法来高效发现频繁项集

## 构建FP树

### 创建一个包含两个子节点的单节点，disp()用于展示 

In [93]:
import fpGrowth
from imp import reload 
reload(fpGrowth)
rootNode = fpGrowth.treeNode('pyramid',9,None)
rootNode.children['eye']=fpGrowth.treeNode('eye',13,None)
rootNode.children['phoenix']=fpGrowth.treeNode('phoenix',3,None)
rootNode.disp()

   pyramid   9
     eye   13
     phoenix   3


### 导入数据，并进行初始化处理

In [94]:
simpDat = fpGrowth.loadSimpDat()
initSet = fpGrowth.createInitSet(simpDat)
simpDat

[['r', 'z', 'h', 'j', 'p'],
 ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
 ['z'],
 ['r', 'x', 'n', 'o', 's'],
 ['y', 'r', 'x', 'z', 'q', 't', 'p'],
 ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]

In [117]:
list(initSet)

Wall time: 0 ns
Wall time: 0 ns


### 构建FP树

In [101]:
from imp import reload
reload(fpGrowth)
myFPtree, myHeaderTab = fpGrowth.createTree(initSet,3)
myFPtree.disp()
myFPtree

['z', 'x', 's', 't', 'y']
[('x', 4), ('s', 3), ('t', 3), ('z', 5), ('y', 3)]
['z', 'x', 'r', 't', 'y']
[('z', 5), ('r', 3), ('t', 3), ('x', 4), ('y', 3)]
['x', 's', 'r']
[('s', 3), ('x', 4), ('r', 3)]
['z', 'x', 't', 's', 'y']
[('x', 4), ('z', 5), ('t', 3), ('s', 3), ('y', 3)]
['z', 'r']
[('z', 5), ('r', 3)]
['z']
[('z', 5)]
   Null Set   1 None
     z   5 None
       x   3 <fpGrowth.treeNode object at 0x0000025A28FC0400>
         s   1 <fpGrowth.treeNode object at 0x0000025A28FC03C8>
           t   1 <fpGrowth.treeNode object at 0x0000025A28FC0630>
             y   1 <fpGrowth.treeNode object at 0x0000025A28FC06A0>
         t   1 None
           s   1 None
             y   1 None
         r   1 <fpGrowth.treeNode object at 0x0000025A28FC0438>
           t   1 <fpGrowth.treeNode object at 0x0000025A28FC04A8>
             y   1 <fpGrowth.treeNode object at 0x0000025A28FC04E0>
       r   1 None
     x   1 None
       s   1 <fpGrowth.treeNode object at 0x0000025A28FC0470>
         r   1 <

### 抽取条件模式基

In [106]:
from imp import reload
reload(fpGrowth)
fpGrowth.findPrefixPath('r',myHeaderTab['y'][1])

frozenset({'z', 's', 't', 'x'})
frozenset({'z', 't', 'x', 'r'})
frozenset({'z', 's', 't', 'x'})


{frozenset({'r', 't', 'x', 'z'}): 1, frozenset({'s', 't', 'x', 'z'}): 1}

### 构建条件FP树

In [98]:
freqItems = []
fpGrowth.mineTree(myFPtree,myHeaderTab,3,set([]),freqItems)
freqItems

['x']
[('x', 3)]
['x']
[('x', 3)]
['x']
[('x', 3)]
conditional tree for:  {'s'}
   Null Set   1
     x   3
['z', 'x']
[('z', 3), ('x', 3)]
['z', 'x']
[('z', 3), ('x', 3)]
['z', 'x']
[('z', 3), ('x', 3)]
conditional tree for:  {'t'}
   Null Set   1
     z   3
       x   3
['z']
[('z', 3)]
conditional tree for:  {'t', 'x'}
   Null Set   1
     z   3
['z']
[('z', 3)]
conditional tree for:  {'x'}
   Null Set   1
     z   3


[{'s'},
 {'s', 'x'},
 {'r'},
 {'t'},
 {'t', 'z'},
 {'t', 'x'},
 {'t', 'x', 'z'},
 {'y'},
 {'x'},
 {'x', 'z'},
 {'z'}]

## 示例：从新闻站点点击流中挖掘

### 从kosarak.dat读取100万条记录

In [128]:
from imp import reload
reload(fpGrowth)
%time parsedDat = [line.split() for line in open('kosarak.dat').readlines()]
parsedDat[:10]

Wall time: 2.82 s


[['1', '2', '3'],
 ['1'],
 ['4', '5', '6', '7'],
 ['1', '8'],
 ['9', '10'],
 ['11', '6', '12', '13', '14', '15', '16'],
 ['1', '3', '7'],
 ['17', '18'],
 ['11', '6', '19', '20', '21', '22', '23', '24'],
 ['1', '25', '3']]

### 构建FP树，只用了几秒，足见算法威力 

In [ ]:
%time initSet = fpGrowth.createInitSet(parsedDat)
%time myFPtree, myHeaderTab = fpGrowth.createTree(initSet,10000)
myHeaderTab

Wall time: 4.18 s


### 条件FP树的构建只用了1毫秒，查看频繁项集 

In [123]:
myFreqList = []
%time fpGrowth.mineTree(myFPtree,myHeaderTab,100000,set([]),myFreqList)
len(myFreqList)
myFreqList

conditional tree for:  {'1'}
   Null Set   1 None
     6   107404 None
conditional tree for:  {'3'}
   Null Set   1 None
     11   9718 None
     6   186289 None
       11   117401 <fpGrowth.treeNode object at 0x0000025A4BC1A8D0>
conditional tree for:  {'11', '3'}
   Null Set   1 None
     6   117401 None
conditional tree for:  {'11'}
   Null Set   1 None
     6   261773 None
Wall time: 1 ms


[{'1'},
 {'1', '6'},
 {'3'},
 {'11', '3'},
 {'11', '3', '6'},
 {'3', '6'},
 {'11'},
 {'11', '6'},
 {'6'}]